In [177]:
import pandas as pd
import numpy as np

In [178]:
data = pd.read_csv('../input/orange_small_churn_data.csv', delimiter =',')
data['labels'] = pd.read_csv('../input/orange_small_churn_labels.csv', header=None)
#конвертируем колонку labels в int
data = data.astype({'labels': 'int32'})
#заменим все -1 в целевой переменной на 0
data['labels'] = data['labels'].map({1: 1, -1: 0})

In [179]:
from sklearn.model_selection import train_test_split
data, holdOut = train_test_split(data, test_size=0.15, random_state=42)

In [180]:
#на этой выборке будем обучать модели. переменная holdOut - это hold out, ее мы не будем использовать ни для обучения 
#ни для оценки baseline моделей
data.shape

In [181]:
#отделим столбец с целевой переменной
target = np.array(data.iloc[:,-1])
target.shape

(34000,)

In [182]:
#подготовим даные:
#выделим категориальные признаки (для baseline решения, возможно будет достаточно числовых)
numericalVarCount = 190
categorialVarCount = 40

data_num = data.iloc[:, 0:numericalVarCount]
#удалим числовые признаки, содержащие слишком большое количество NaN - значений
threshold = 0.7
NaN_frac = data_num.isna().sum(axis = 0)/data_num.shape[0]
numVarsToStay = list(NaN_frac[NaN_frac < threshold].index)
data_num = data_num.loc[:,numVarsToStay]
#Перед построением моделей, подготовим данные: заменим NaN на медианные значения,
medians = data_num.median()
data_num.fillna(medians, inplace=True)
#выполним стандартизацию числовых признаков
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_num = scaler.fit_transform(data_num)
data_num.shape


(34000, 42)

In [183]:

# #обработаем категориальные признаки методикой one-hot-encoding
from sklearn.feature_extraction import DictVectorizer as DV
data_cat = data.iloc[:, numericalVarCount:-1]

data_cat_oh = pd.get_dummies(data_cat, dummy_na=True, drop_first=True)
NaN_frac = data_cat.isna().sum(axis = 0)/data_cat.shape[0]
NaN_frac
threshold = 0.1
NaN_frac = data_cat.isna().sum(axis = 0)/data_num.shape[0]
catVarsToStay = list(NaN_frac[NaN_frac < threshold].index)
data_cat = data_cat.loc[:,catVarsToStay]
data_cat = data_cat.fillna('NA').astype(str)

#Подсчитаем количество уникальных значений в категориальных признаках, от этого будет зависеть способ кодировки
unique_counts = []
for c in data_cat.columns:
    unique_counts.append(data_cat[c].dropna().unique().shape[0])
cat_unique = pd.DataFrame()
cat_unique['unique_counts'] = unique_counts
cat_unique.index = data_cat.columns
cat_unique.sort_values(by='unique_counts', ascending=False)
cat_unique


,unique_counts
Var192,347
Var193,47
Var195,22
Var196,4
Var197,220
Var198,3587
Var199,3929
Var202,5385
Var203,6
Var204,100


In [184]:
cat_feat_for_OHE = list(cat_unique[cat_unique['unique_counts'] < 50].index)
cat_feat_for_OHE
encoder = DV(sparse = False)
data_cat_oh = encoder.fit_transform(data_cat[cat_feat_for_OHE].T.to_dict().values())

In [185]:
data_cat_oh.shape

(34000, 178)

In [186]:
pd.isna(data_cat_oh).sum().sum()

0

In [187]:
#объединяем обработанные числовые и категориальные признаки
data_all = np.hstack((data_num,data_cat_oh))
data_all.shape

(34000, 220)

In [190]:
#будем использовать следующие модели для baseline решения:
#RandomForestClassifier, RidgeClassifier и SGDClassifier
#ввиду несбалансированности выборок везде используем class_weight='balanced'
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
#from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

In [191]:
from sklearn.model_selection import cross_val_score

clf = SGDClassifier(loss = 'log', class_weight='balanced', max_iter=1000, tol=1e-4, alpha=0.01, random_state=42)
# scores = cross_val_score(clf, data_all, target, cv=5, scoring = 'f1')
# scores.mean()

# clf = LogisticRegression(class_weight='balanced', penalty='l1')
# scores = cross_val_score(clf, data_all, target, cv=5, scoring = 'f1')
# scores.mean()


# from sklearn.neural_network import MLPClassifier
# clf = MLPClassifier(hidden_layer_sizes=(128,64,32,16,8),learning_rate_init = 0.01, max_iter = 500)
# scores = cross_val_score(clf, data_num, target, cv=5, scoring = 'f1')
# scores.mean()

# from sklearn.ensemble import GradientBoostingClassifier
# clf = GradientBoostingClassifier(n_estimators=10, learning_rate=1, max_depth=11, random_state=0)
# scores = cross_val_score(clf, data_num, target, cv=5, scoring = 'f1')
# scores.mean()
# clf = SVC(gamma='auto',kernel = 'linear')
# scores = cross_val_score(clf, data_all, target, cv=3, scoring = 'f1')

# clf = LinearSVC(class_weight = 'balanced', random_state = 42)
# scores = cross_val_score(clf, data_all, target, cv=5, scoring = 'roc_auc')
# scores.mean()

# clf = RandomForestClassifier(n_estimators=100, max_depth=4, class_weight ='balanced', random_state=42)
# scores = cross_val_score(clf, data_num, target, cv=3, scoring = 'roc_auc')
# scores.mean()
# import xgboost as xgb
# clf = xgb.XGBClassifier()
# scores = cross_val_score(gbm, data_all, target, cv=3, scoring = 'f1')
# scores

0.2016434155048039

In [192]:
# import xgboost as xgb
# clf = xgb.XGBClassifier()
# scores = cross_val_score(clf, data_num, target, cv=3, scoring = 'roc_auc')
# scores.mean()

In [193]:
#загружаем тестовую выборку:
test = pd.read_csv('../input/orange_small_churn_test_data.csv',delimiter = ',')
test.drop(columns = 'ID', inplace=True)
test.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
0,NaN,NaN,NaN,NaN,NaN,1225.0,7.0,NaN,NaN,NaN,...,zCkv,APgdzOv,jySVZNlOJy,NaN,ELof,xb3V,6fzt,Zy3gnGM,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,896.0,14.0,NaN,NaN,NaN,...,oslk,IIvC99a,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,791.0,7.0,NaN,NaN,NaN,...,oslk,6YSocsg,LM8l689qOp,NaN,kG3k,rgKb,RAYp,F2FyR07IdsN7I,mj86,NaN
3,NaN,NaN,NaN,NaN,NaN,2296.0,7.0,NaN,NaN,NaN,...,oslk,5nQ7A2G,jySVZNlOJy,NaN,kG3k,rgKb,RAYp,F2FyR07IdsN7I,am7c,NaN
4,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,...,oslk,MI8s5nE,LM8l689qOp,NaN,NaN,7P5s,RAYp,F2FyR07IdsN7I,NaN,NaN


In [194]:
#выполняем на ней ту же обработку, что для набора обучения
test_num = test.iloc[:, 0:numericalVarCount]
#выкинем признаки, которые выкидывали при обучении
test_num = test.loc[:,numVarsToStay]
#заполним NaN медианными значениями train!!! набора
test_num.fillna(medians, inplace=True)
#выполним стандартизацию, с теми же параметрами, что при обучении:
test_num = scaler.transform(test_num)
test_cat = test.iloc[:, numericalVarCount:-1]
test_cat = test_cat.loc[:,catVarsToStay]
test_cat = test_cat.fillna('NA').astype(str)
test_cat_oh = encoder.transform(test_cat[cat_feat_for_OHE].T.to_dict().values())
test_all = np.hstack((test_num,test_cat_oh))
test_all.shape

(10000, 220)

In [196]:
#clf = SGDClassifier(loss = 'log', class_weight='balanced', max_iter=1000, tol=1e-4, alpha=0.1)
clf.fit(data_all, target)

# clf = xgb.XGBClassifier(scale_pos_weight=0.90)
# clf.fit(data_num, target)

SGDClassifier(alpha=0.01, average=False, class_weight='balanced',
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=42, shuffle=True, tol=0.0001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [197]:
#testPredictions = clf.predict(test_num)
testPredictions = clf.predict_proba(test_all)[:,1]

In [198]:
#записываем ответ в файл
ansData = {'Id':test.index, 'result':testPredictions} 
ans = pd.DataFrame(ansData)
ans.to_csv('ans.csv', index= False)